Approche : mélanger analyse lexicale et par apprentissage.

1. Analyse lexicale des tweets :
    - POS tagging, lemmisation
    - Traduction des mots en Anglais pour pouvoir utiliser Sentiwordnet et obtenir la polarité des mots

2. Application d'un modèle d'apprentissage supervisé :
    - HMM, SVM, etc...

Lien vers les POS tags en français : http://www.cis.uni-muenchen.de/~schmid/tools/TreeTagger/data/french-tagset.html

Etudier la possibilité d'ajout de lexiques d'opinion en Français : http://alpage.inria.fr/~sagot/wolf.html, http://sites.univ-provence.fr/wpsycle/outils_recherche/liwc/FrenchLIWCDictionary_V1_1.dic, http://sites.univ-provence.fr/~wpsycle/outils_recherche/outils_recherche.html#emotaix

Voir l'approche compositionnelle : Moilanen 2007

In [4]:
import pprint
import treetaggerwrapper
import time

import pymongo as pym
#import nltk.data
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import TreebankWordTokenizer
import stop_words
#from nltk.stem import *
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.svm import SVC, LinearSVC
import pandas as pd
import numpy as np

In [4]:
stops = set(['rt','ds','qd','ss','ns','vs','nn','amp','gt','gd','gds','tt','pr','ac','mm', 'qu',
            '``', 'ni', 'ca', 'le', 'les', ' ', 'si', '$', '^', 'via', 'ils','pour','une','que','quel']
#            + list('@ن%£€‘:&;')
            + list('abcdefghijklmnopqrstuvwxyzà'))

#            stop_words.get_stop_words(language='fr')+stopwords.words('french'))

In [1]:
stops = set(list('abcdefghijklmnopqrstuvwxyzà'))
# TODO: améliorer la liste de stop words

In [2]:
def mongo_to_df(collection, n_last_tweets=0, retweet=False):
    tweets = collection.find(filter={'text':{'$exists':True}}, 
                             projection={'_id':False}).sort('$natural',-1).limit(n_last_tweets)
    df = pd.DataFrame()
    listTweets, listCandidats, listSentiments = [], [], []
    
    for t in tweets: 
        if not retweet: # filtrage des retweets
            if 'rt @' in t['text']:
                continue

        if t['text']: # test si liste non vide
            listTweets.append(t['text'])
            try:
                listCandidats.append(t['candidat'])
            except:
                listCandidats.append(None)
            
            try:
                listSentiments.append(t['sentiment'])
            except:
                listSentiments.append(None)
    
    df['text'], df['candidat'], df['sentiment'] = listTweets, listCandidats, listSentiments
    return df

def regex_filter(text):
    text = re.sub(r'\w*…', '', text) # mot tronqué par Twitter
    text = re.sub(r'(?:htt)\S*', '', text) # retrait des liens http
    text = re.sub(r'\n', ' ', text) # retrait des sauts de ligne
    text = re.sub(r'\xad', '-', text)
    text = re.sub(r'@\w*', '', text) # retrait des mentions @ (ne détecte pas @XXX@...)
    text = re.sub(r'\.{3,}', '...', text) # ....... => points de suspension
    text = re.sub(r'(?=\.\w)(\.)', '. ', text) # remplacer un point entre deux mots 'A.B' par 'A. B'
    #text = re.sub(r'\[a-zA-Z]*(?!\S)', '', text) # retrait de ce qui n'est pas un mot
    #text = re.sub(r'^rt.*: ', '', text) # retrait de la mention retweet
    #text = re.sub(r'\d', '', text) # retrait des chiffres
    #text = re.sub(r',;!?\/\*(){}«»', ' ', text)
    #text = re.sub('|'.join(['’', '_', '/', '-', '\'', '“', '\.']), ' ', text)
    #text = re.sub('|'.join([elem + '\'' for elem in 'cdjlmnst']), '', text) # apostrophes
    return text

def process_texts(list_of_texts):
    # Processing the tweets (POS tagging, lemmatization, spellchecking)
    tagger = treetaggerwrapper.TreeTagger(TAGLANG='fr')
    list_of_processed_texts = []
    
    for text in list_of_texts:
        text = regex_filter(text)
        
        # TODO: (optionnel) retirer les # avant d'utiliser TreeTagger
        # puis les remettre avec le tag HASH|
        
        # TODO: correction orthographique des tweets
        
        tags = tagger.tag_text(text)
        try:
            tagged_text = ['{}|{}'.format(t.split('\t')[1], t.split('\t')[2]) for t in tags
                           if t.split('\t')[2] not in stops] # ou bien filtrer sur le POS tag
        except:
            tagged_text = ['ERREUR']
        
        # append les HASH|#...
        # TODO: (optionnel) gérer les accents sur les mots
        list_of_processed_texts.append(tagged_text)
    return list_of_processed_texts

#### Test du TreeTagger

In [22]:
tagger = treetaggerwrapper.TreeTagger(TAGLANG='fr')
tags = tagger.tag_text('exemple de texte à taguer')
print(tags)
tagged_text = ['{}|{}'.format(t.split('\t')[1], t.split('\t')[2]) for t in tags]
pprint.pprint(tagged_text)

['exemple\tNOM\texemple', 'de\tPRP\tde', 'texte\tNOM\ttexte', 'à\tPRP\tà', 'taguer\tNOM\ttaguer']
['NOM|exemple', 'PRP|de', 'NOM|texte', 'PRP|à', 'NOM|taguer']


### Loading the tweets

In [26]:
client = pym.MongoClient('localhost',27017)

#### Base annotée manuellement

In [31]:
collection = client.tweet.train
print('{} tweets in the manual train set.'.format(collection.count()))
df_tweets = mongo_to_df(collection, n_last_tweets=0, retweet=True)
print(df_tweets['sentiment'].value_counts())
print(df_tweets['candidat'].value_counts())
df_tweets.head(5)

9912 tweets in the manual train set.
-1.0    5275
 0.0    3437
 1.0    1199
Name: sentiment, dtype: int64
Series([], Name: candidat, dtype: int64)


,text,candidat,sentiment
0,"ben voyons donc ! ""par erreur"" aussi ils nous ...",None,-1.0
1,l'écologie version macron : les contradictions...,None,-1.0
2,"@ericwoerth : ""françois #fillon est audible : ...",None,1.0
3,fillon fait honte au monde entier mais il s ac...,None,-1.0
4,@gg_rmc @bayrou la girouette de la politique-l...,None,-1.0


#### Base annotée automatiquement, sur la base des hashtags (uniquement des tweets positifs)

In [28]:
collection = client.tweet.labelised
print('{} tweets in the auto train set.'.format(collection.count()))
df_tweets_auto = mongo_to_df(collection, n_last_tweets=0, retweet=False)
print(df_tweets_auto['sentiment'].value_counts())
print(df_tweets_auto['candidat'].value_counts())
df_tweets_auto.head(5)

1421 tweets in the auto train set.
1    1418
Name: sentiment, dtype: int64
fillon       299
macron       289
le pen       288
hamon        274
melenchon    268
Name: candidat, dtype: int64


,text,candidat,sentiment
0,même s'il veut faire croire qu'il l'adoucit le...,melenchon,1
1,si #fillon pas présent second tour #mlp sera m...,melenchon,1
2,#fillon et #macron sont au coude à coude dans ...,melenchon,1
3,"@edfofficiel votre pub est à chier, où avait v...",melenchon,1
4,@sofiakkar faut pas pousser #fillon parle de...,melenchon,1


### Train & predict

In [29]:
# Tweet feature extraction
hashtag = [t.count('#') for t in df_tweets['text']]
links = [t.count('http') for t in df_tweets['text']]
at = [t.count('@') for t in df_tweets['text']]
n_car = [np.log(len(t)) for t in df_tweets['text']]
n_words = [np.log(len(t.split(' '))) for t in df_tweets['text']]

try:
    hashtag.extend([t.count('#') for t in df_tweets_auto['text']])
    links.extend([t.count('http') for t in df_tweets_auto['text']])
    at.extend([t.count('@') for t in df_tweets_auto['text']])
    n_car.extend([np.log(len(t)) for t in df_tweets_auto['text']])
    n_words.extend([np.log(len(t.split(' '))) for t in df_tweets_auto['text']])
except:
    pass

# a = t['text'].count('!')
# b = t['text'].count('?')
# c = t['text'].count('#')
# d = t['text'].count('"')
# e = t['text'].count('http')
# f = t['text'].count(':')
# g = t['text'].count('»')
# h = t['text'].count('@')

# Tweet processing
tweet_list = process_texts(df_tweets['text'])
try:
    tweet_list.extend(process_texts(df_tweets_auto['text']))
except:
    pass
print('TreeTagger a renvoyé {} erreur(s).'.format(tweet_list.count('ERREUR')))

# Building feature matrix
vectorizer = TfidfVectorizer(strip_accents=None, analyzer='word', decode_error='strict',
                            use_idf=False, norm=None, binary=False, min_df=1, max_df=1.0, ngram_range=(1,1))
tfidf = vectorizer.fit_transform([' '.join(tweet) for tweet in tweet_list])
X = pd.DataFrame(tfidf.toarray())
X['#'], X['http'], X['@'], X['n_car'], X['n_words'] = hashtag, links, at, n_car, n_words
print(X[:5])

try:
    y = pd.concat([df_tweets['sentiment'], df_tweets_auto['sentiment']], axis=0)
except:
    y = df_tweets['sentiment']

TreeTagger a renvoyé 0 erreur(s).
     0    1    2    3    4    5    6    7    8    9    ...     11797  11798  \
0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0    ...       0.0    0.0   
1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0    ...       0.0    0.0   
2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0    ...       0.0    0.0   
3  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0    ...       0.0    0.0   
4  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0    ...       0.0    0.0   

   11799  11800  11801  #  http  @     n_car   n_words  
0    0.0    0.0    0.0  0     0  3  4.828314  2.833213  
1    0.0    0.0    0.0  0     1  1  4.753590  2.772589  
2    0.0    0.0    0.0  1     2  0  4.919981  2.890372  
3    0.0    0.0    0.0  1     1  0  4.919981  3.091042  
4    0.0    0.0    0.0  0     1  0  4.465908  2.484907  

[5 rows x 11807 columns]


In [43]:
# Building train & test sets
X_train,X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
n_samples, vocabulaire = X.shape

print("Répartition dans le dataset de train ({} tweets) : \n".format(len(y_train)),
      "\tNégatif :", np.abs(np.sum(y_train[y_train == -1])/len(y_train)),
      "%\n\tPositif :", np.abs(np.sum(y_train[y_train == 1])/len(y_train)),"%")
print("Répartition dans le dataset de test ({} tweets) : \n".format(len(y_test)),
      "\tNégatif :", np.abs(np.sum(y_test[y_test == -1])/len(y_test)),
      "%\n\tPositif :", np.abs(np.sum(y_test[y_test == 1])/len(y_test)),"%")
print('Tweets : ' + str(n_samples) + ' / ' + 'N-grams : ' + str(vocabulaire))

# Choise of models
clf = LogisticRegression(penalty='l2', C=1., max_iter=2000, class_weight='balanced', multi_class='ovr')
#clf = SVC(C=1.0, class_weight='balanced', max_iter=2000, kernel='linear', decision_function_shape='ovr')
#clf = LinearSVC(C=.5, dual=True, class_weight='balanced')
#clf = MultinomialNB(alpha=1.0, fit_prior=True, class_prior=None)

# Fit & predict
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
print('\nScore', np.sum(predictions == y_test) / len(predictions))

print('Répartition des prédictions : \n',
      '\tNégatif :', np.abs(np.sum(predictions[predictions == -1])/len(predictions)),
      '%\n\tPositif :', np.abs(np.sum(predictions[predictions == 1])/len(predictions)), '%')

# TODO: ajouter matrice de confusion, score f1

Répartition dans le dataset de train (9054 tweets) : 
 	Négatif : 0.464877402253 %
	Positif : 0.232273028496 %
Répartition dans le dataset de test (2264 tweets) : 
 	Négatif : 0.468197879859 %
	Positif : 0.225265017668 %
Tweets : 11318 / N-grams : 11807

Score 0.67093639576
Répartition des prédictions : 
 	Négatif : 0.469081272085 %
	Positif : 0.191696113074 %
